In [7]:
import os
import sys
from pathlib import Path

# FOR LOCAL USE THIS LINES
# current = Path.cwd()
# src_path = current / "src" if (current / "src").exists() else current.parent

# FOR COLAB USE THIS LINE INSTEAD
BRANCH_NAME = "classification-resnet-refactoring"  # Change this to switch branches
!git clone -b {BRANCH_NAME} https://github.com/MatteoCamillo-code/GeoLoc-CVCS.git
!cd /content/GeoLoc-CVCS && git pull origin {BRANCH_NAME} && cd ..
src_path = Path("/content/GeoLoc-CVCS/src").resolve()

sys.path.insert(0, str(src_path))

from utils.paths import find_project_root

# Set working directory and sys.path properly
project_root = find_project_root(src_path)
data_dir = project_root / "data"
history_dir = project_root / "outputs" / "history"
os.chdir(project_root)
sys.path.insert(0, str(project_root / "src"))
print("CWD:", Path.cwd())

fatal: destination path 'GeoLoc-CVCS' already exists and is not an empty directory.
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 460 bytes | 460.00 KiB/s, done.
From https://github.com/MatteoCamillo-code/GeoLoc-CVCS
 * branch            classification-resnet-refactoring -> FETCH_HEAD
   6b49589..92a9039  classification-resnet-refactoring -> origin/classification-resnet-refactoring
Updating 6b49589..92a9039
Fast-forward
 src/scene_classification/{ => resnet50}/merger_utils.py | 0
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename src/scene_classification/{ => resnet50}/merger_utils.py (100%)
CWD: /content/GeoLoc-CVCS


In [ ]:
import os
import glob
import time
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader

from src.scene_classification.resnet50.classifier_model import SceneClassifier
from src.scene_classification.resnet50.data_utils import ProjectPaths, SceneDataset, collate_fn
from src.scene_classification.resnet50.merger_utils import update_original_file

# 1. SETUP
paths = ProjectPaths() 
classifier = SceneClassifier()

# 2. DATA LOADING
image_files = glob.glob(os.path.join(paths.base_image_path, "**", "*.jpg"), recursive=True)
loader = DataLoader(SceneDataset(image_files, classifier.transform), 
                    batch_size=64, num_workers=2, collate_fn=collate_fn)

# 3. BULK CLASSIFICATION (Function call instead of local loop)
results_list = classifier.batch_classify(loader)
df_results = pd.DataFrame(results_list)

# 4. MERGE AND SAVE
update_original_file(df_results, paths.original_train_csv, "train_with_predictions.csv")
update_original_file(df_results, paths.original_test_csv, "test_with_predictions.csv")

# 5. SINGLE IMAGE CLASSIFICATION
label = classifier.classify_one_image(image_files[0])
print(f"Prediction for {os.path.basename(image_files[0])}: {label}")

In [ ]:
# 2. DATA LOADING
image_files = glob.glob(os.path.join(paths.base_image_path, "**", "*.jpg"), recursive=True)
loader = DataLoader(SceneDataset(image_files, classifier.transform), 
                    batch_size=64, num_workers=2, collate_fn=collate_fn)

Found 0 images.


0it [00:00, ?it/s]

Process completed in 0.07s


In [ ]:
# 3. BULK CLASSIFICATION (Function call instead of local loop)
results_list = classifier.batch_classify(loader)
df_results = pd.DataFrame(results_list)

In [ ]:
# 4. MERGE AND SAVE
update_original_file(df_results, paths.original_train_csv, "train_with_predictions.csv")
update_original_file(df_results, paths.original_test_csv, "test_with_predictions.csv")

# 5. SINGLE IMAGE CLASSIFICATION
label = classifier.classify_one_image(image_files[0])
print(f"Prediction for {os.path.basename(image_files[0])}: {label}")